In [57]:
import pandas as pd
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf 
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder
import tensorflow.keras as keras
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

In [58]:
review_df = pd.read_pickle('food_review_places.pickle')

In [59]:
review_df['rating'].value_counts()

5.0    34487
4.0    19041
3.0     8118
2.0     5010
1.0     3539
Name: rating, dtype: int64

In [60]:
review_df

,gPlusPlaceId,rating,reviewerName,reviewText,categories,unixReviewTime,reviewTime,gPlusUserId,name,price,...,hours,phone,closed,gps,lat,long,open_days,Dinner,Lunch,Breakfast
0,112181651134020562716,5.0,jason wagner,stop dinner mahi mahi special fantast servic g...,"[european restaurant, italian restaurant]",2012-10-18 21:56:05,2012-10-18,100000196778399872657,Pasquales Trattoria,None,...,"[[Monday, [['Closed']]], [Tuesday, [['11:30 am...",(905) 338-9090,False,"[43.440331, -79.672851]",43.440331,-79.672851,6.0,1.0,0.0,0.0
1,109391653021436219474,5.0,heather goldsworthy,best coffe collingwood,[cafe],2013-09-12 18:38:15,2013-09-12,100000427343266187570,Espresso Post,None,...,None,(705) 446-1740,False,"[44.499252, -80.216587]",44.499252,-80.216587,NaN,NaN,NaN,NaN
2,109391653021436219474,5.0,nicholas clayton,locat old post offic outlet histor pedigre bes...,[cafe],2013-05-04 01:35:12,2013-05-03,104975196938413074237,Espresso Post,None,...,None,(705) 446-1740,False,"[44.499252, -80.216587]",44.499252,-80.216587,NaN,NaN,NaN,NaN
3,110864889964667973890,5.0,chris johnson,get right everi time love guy,"[delivery restaurant, event planner, business ...",2011-09-15 21:54:03,2011-09-15,100000524810171549476,"Delivered Dish | Portland, OR",$$,...,"[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(503) 239-0100,False,"[45.470342, -122.747085]",45.470342,-122.747085,7.0,1.0,0.0,0.0
4,110864889964667973890,1.0,allison lea,use servic order even show 1 hour 20 minut pro...,"[delivery restaurant, event planner, business ...",2013-11-09 10:28:01,2013-11-08,100537426867747151661,"Delivered Dish | Portland, OR",$$,...,"[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(503) 239-0100,False,"[45.470342, -122.747085]",45.470342,-122.747085,7.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70190,113840395279665001642,5.0,ture jackson,great,[cafe],2012-06-10 03:18:34,2012-06-09,106224892970344797880,Second Cup,None,...,"[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",(604) 514-5085,True,"[49.121205, -122.668297]",49.121205,-122.668297,7.0,1.0,0.0,1.0
70191,108887431842899649801,4.0,tomasz w,surprisingli good mere 14 buck get larg pizza ...,[delivery restaurant],2013-11-16 05:42:46,2013-11-15,106224899821829244630,Little Caesars Pizza,$$$,...,"[[Monday, [['11:00 am--11:00 pm']]], [Tuesday,...",(604) 584-0090,False,"[49.176368, -122.867572]",49.176368,-122.867572,7.0,1.0,1.0,0.0
70192,112463911517644715471,2.0,adrian ludwin,busi march 2013,"[cafe, dessert shop, ice cream shop]",2013-03-08 18:03:34,2013-03-08,106225840000504273299,Baskin-Robbins Ice Cream,$$,...,None,(416) 698-3103,True,"[43.669867, -79.301196]",43.669867,-79.301196,NaN,NaN,NaN,NaN
70193,105703129044638346837,4.0,timothy decker,great food tim owner btw differ tim lol work h...,[bar],2013-08-15 05:50:22,2013-08-14,106226813159080833866,Rainbow Bar,None,...,"[[Monday, [['7:00 am--4:00 am']]], [Tuesday, [...",(989) 725-9880,False,"[42.997547, -84.182386]",42.997547,-84.182386,7.0,1.0,1.0,1.0


In [61]:
enc = OneHotEncoder()
reviews = list(review_df['reviewText'])
ratings = np.array(list(review_df['rating']))
# one-hot encode the ratings
ratings = enc.fit_transform(ratings.reshape(-1, 1)).toarray()

In [62]:
split = 0.8
train_len = int(split * len(reviews))
train_reviews = np.array(reviews[:train_len])
train_ratings = np.array(ratings[:train_len])
test_reviews = np.array(reviews[train_len:])
test_ratings = np.array(ratings[train_len:])

In [63]:
max_size = 1000
vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=max_size, output_mode='int', output_sequence_length=20)
vectorize_layer.adapt(train_reviews)

D:\Anaconda\envs\py36gpu\lib\site-packages\tensorflow\python\keras\layers\preprocessing\text_vectorization.py:338: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(preprocessed_data.to_list())


In [64]:
inputs = tf.keras.layers.Input(shape=(1, ), dtype=tf.string, name='text')
outputs = vectorize_layer(inputs)
model = tf.keras.Model(inputs, outputs)
# vocab = np.array(vectorize_layer.get_vocabulary())

In [65]:
encoded_train_reviews = model.predict(train_reviews)
encoded_test_reviews = model.predict(test_reviews)

In [72]:
inputs = tf.keras.Input(shape=(20,), dtype="int64")
x = layers.Embedding(input_dim=1001, output_dim=256,)(inputs)
x = layers.Dropout(0.2)(x)
x = layers.Conv1D(32, 2, padding="valid", activation="relu", strides=1)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.2)(x)
predictions = layers.Dense(5, activation="softmax", name="predictions")(x)
model = tf.keras.Model(inputs, predictions)

In [73]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()
history = model.fit(encoded_train_reviews, train_ratings, validation_split=0.2, epochs=10)

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 20)]              0         
_________________________________________________________________
embedding_14 (Embedding)     (None, 20, 256)           256256    
_________________________________________________________________
dropout_22 (Dropout)         (None, 20, 256)           0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 19, 32)            16416     
_________________________________________________________________
global_max_pooling1d_13 (Glo (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_23 (Dropout)         (None, 64)                0  

In [ ]:
score, acc = model.evaluate(encoded_test_reviews, test_ratings)